# Simulation Ramp

In [3]:
#-----Import essential packages

from __future__ import division
import hoomd
import hoomd.md

#-----Deine some important variables.

p_0 = 1.6;
p_1 = 1.8;
t_max  = 5.0;
ramp_steps = 1e5;

#-----Create a simulation context.

hoomd.context.initialize("--mode=cpu", memory_traceback = True);

#-----Initialize from a gsd file.

system = hoomd.init.read_gsd("T_CM_5.0_P_1.6_ramp.gsd",frame = -1)

#-----Define each mesogen in the local frame of reference in the rigid body.

rigid = hoomd.md.constrain.rigid();
rigid.set_param('M', 
                types=['A']*8,
                positions=[(-4,0,0),(-3,0,0),(-2,0,0),(-1,0,0),
                           (1,0,0),(2,0,0),(3,0,0),(4,0,0)]);

#-----Create mesogens.

rigid.create_bodies();

#-----Define the potential energy.

nl = hoomd.md.nlist.cell();

lj = hoomd.md.pair.lj(r_cut = 3.5, nlist = nl)
lj.set_params(mode='shift')

#-----Define the interaction between each constituent in the mix.

lj.pair_coeff.set('M','M', epsilon = 1.0, sigma = 1.0)
lj.pair_coeff.set('A','A', epsilon = 1.0, sigma = 1.0)
lj.pair_coeff.set('M','A', epsilon = 1.0, sigma = 1.0)

#-----Select an standard integrator.

hoomd.md.integrate.mode_standard(dt = 0.005);

#------Define the group on wich we will integrate the equations of motion.

mesogens = hoomd.group.rigid_center();

notice(2): Group "all" created containing 1000 particles
notice(2): constrain.rigid(): Creating 1000 rigid bodies (adding 8000 particles)
notice(2): Group "rigid_center" created containing 1000 particles


In [4]:
#----Integrate using NPT

press_1 = hoomd.variant.linear_interp(points = [(0.0,p_0), (ramp_steps, p_1)])

npt = hoomd.md.integrate.npt(group = mesogens, kT = t_max, tau = 1.0, tauP = 13.0, P = press_1);
npt.randomize_velocities(seed = 42)

#-----Save data in a .gds and .log file.

log_file = "T_" + str(t_max) + "_P_" + str(p_1) + "_ramp.log"
gsd_file = "T_" + str(t_max) + "_P_" + str(p_1) + "_ramp.gsd" 
meso_gsd_file = "T_CM_" + str(t_max) + "_P_" + str(p_1) + "_ramp.gsd"

log = hoomd.analyze.log(filename = log_file,
                         quantities = ['num_particles',
                                     'ndof',
                                     'translational_ndof',
                                     'rotational_ndof',
                                     'potential_energy',
                                     'kinetic_energy',
                                     'translational_kinetic_energy',
                                     'rotational_kinetic_energy',
                                     'temperature',
                                     'pressure',
                                      'volume'],
                         period = 1e3,
                         overwrite = True);
gsd = hoomd.dump.gsd(gsd_file,
               period = 1e3,
               group = hoomd.group.all(),
               overwrite = True); 
meso_gsd = hoomd.dump.gsd(meso_gsd_file,
               period = 1e3,
               group = mesogens,
               overwrite = True); 

In [5]:
#-----Run the simulation.

hoomd.run(ramp_steps)

notice(2): -- Neighborlist exclusion statistics -- :
notice(2): Particles with 0 exclusions             : 9000
notice(2): Neighbors included by diameter          : no
notice(2): Neighbors excluded when in the same body: yes
** starting run **
Time 00:00:10 | Step 830094 / 930000 | TPS 9.32421 | ETA 02:58:34
Time 00:00:20 | Step 830223 / 930000 | TPS 12.8954 | ETA 02:08:57
Time 00:00:30 | Step 830382 / 930000 | TPS 15.6972 | ETA 01:45:46
Time 00:00:40 | Step 830532 / 930000 | TPS 14.9047 | ETA 01:51:13
Time 00:00:50 | Step 830662 / 930000 | TPS 12.9921 | ETA 02:07:26
Time 00:01:00 | Step 830783 / 930000 | TPS 12.0834 | ETA 02:16:51
Time 00:01:10 | Step 830918 / 930000 | TPS 13.3462 | ETA 02:03:43
Time 00:01:20 | Step 831074 / 930000 | TPS 15.3967 | ETA 01:47:05
Time 00:01:30 | Step 831229 / 930000 | TPS 15.4745 | ETA 01:46:22
Time 00:01:40 | Step 831340 / 930000 | TPS 10.9059 | ETA 02:30:46
Time 00:01:50 | Step 831437 / 930000 | TPS 9.62989 | ETA 02:50:35
Time 00:02:00 | Step 831532 / 9

**ERROR**: Particle with unique tag 671 is no longer in the simulation box.

**ERROR**: Cartesian coordinates: 
**ERROR**: x: -5.72878 y: 264.268 z: 110.145
**ERROR**: Fractional coordinates: 
**ERROR**: f.x: 0.24856 f.y: 12.0989 f.z: 5.33435
**ERROR**: Local box lo: (-11.3919, -11.3919, -11.3919)
**ERROR**:           hi: (11.3919, 11.3919, 11.3919)


RuntimeError: Error computing cell list

In [ ]:
#------Display some information (density).

system.get_metadata()